In [1]:
import pandas as pd
import sqlite3
import sqlalchemy as sql
import mysql.connector
from sqlalchemy import create_engine
import pymysql

In [2]:
# load first data set
data = pd.read_csv("books.csv") 

In [3]:
data.head()

,Publishing Year,Book Name,Author,language_code,Author_Rating,Book_average_rating,Book_ratings_count,genre,gross sales,publisher revenue,sale price,sales rank,Publisher,units sold
0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903,genre fiction,34160.0,20496.0,4.88,1,HarperCollins Publishers,7000
1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,Intermediate,4.23,145267,genre fiction,12437.5,7462.5,1.99,2,HarperCollins Publishers,6250
2,2015.0,Go Set a Watchman,Harper Lee,eng,Novice,3.31,138669,genre fiction,47795.0,28677.0,8.69,3,"Amazon Digital Services, Inc.",5500
3,2008.0,When You Are Engulfed in Flames,David Sedaris,en-US,Intermediate,4.04,150898,fiction,41250.0,24750.0,7.50,3,Hachette Book Group,5500
4,2011.0,Daughter of Smoke & Bone,Laini Taylor,eng,Intermediate,4.04,198283,genre fiction,37952.5,22771.5,7.99,4,Penguin Group (USA) LLC,4750


In [4]:
# load second data set
reads = pd.read_csv('reads.csv',header=0,encoding = 'unicode_escape')

In [5]:
reads.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count,Unnamed: 10
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPrï¿,4.56,439785960,9.78044E+12,eng,652,1944099,26249,NaN
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPrï¿,4.49,439358078,9.78044E+12,eng,870,1996446,27613,NaN
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPrï¿,4.47,439554934,9.78044E+12,eng,320,5629932,70390,NaN
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,439554896,9.78044E+12,eng,352,6267,272,NaN
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPrï¿,4.55,043965548X,9.78044E+12,eng,435,2149872,33964,NaN


In [6]:
# remove that NaN column
del reads['Unnamed: 10']

In [7]:
del reads['bookID']

In [8]:
reads.rename(columns={'authors':'GoodReads_author'}, inplace=True)

In [9]:
del reads['language_code']

In [10]:
# rename the book name column to title so we can merge on title
data.rename(columns={'Book Name':'title', 'Book_average_rating': 'Amazon_average_rating'}, inplace=True)


In [11]:
reads.rename(columns={'average_rating': 'GoodReads_average_rating'}, inplace=True)

In [12]:
reads.rename(columns={'author': 'GoodReads_author'}, inplace=True)

In [13]:
data.head()

,Publishing Year,title,Author,language_code,Author_Rating,Amazon_average_rating,Book_ratings_count,genre,gross sales,publisher revenue,sale price,sales rank,Publisher,units sold
0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903,genre fiction,34160.0,20496.0,4.88,1,HarperCollins Publishers,7000
1,1987.0,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,Intermediate,4.23,145267,genre fiction,12437.5,7462.5,1.99,2,HarperCollins Publishers,6250
2,2015.0,Go Set a Watchman,Harper Lee,eng,Novice,3.31,138669,genre fiction,47795.0,28677.0,8.69,3,"Amazon Digital Services, Inc.",5500
3,2008.0,When You Are Engulfed in Flames,David Sedaris,en-US,Intermediate,4.04,150898,fiction,41250.0,24750.0,7.50,3,Hachette Book Group,5500
4,2011.0,Daughter of Smoke & Bone,Laini Taylor,eng,Intermediate,4.04,198283,genre fiction,37952.5,22771.5,7.99,4,Penguin Group (USA) LLC,4750


In [14]:
# merge the cleaned data
combined = pd.merge(data, reads, how='right', on=['title'])

In [15]:
combined.head(5)

,Publishing Year,title,Author,language_code,Author_Rating,Amazon_average_rating,Book_ratings_count,genre,gross sales,publisher revenue,...,sales rank,Publisher,units sold,GoodReads_author,GoodReads_average_rating,isbn,isbn13,# num_pages,ratings_count,text_reviews_count
0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903.0,genre fiction,34160.00,20496.000,...,1.0,HarperCollins Publishers,7000.0,Unknown-Burton Raffel,3.43,451527402,9.78045E+12,160,2735,149
1,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903.0,genre fiction,34160.00,20496.000,...,1.0,HarperCollins Publishers,7000.0,Michael Morpurgo-Michael Foreman,3.85,763632066,9.78076E+12,96,43,7
2,1994.0,Midnight in the Garden of Good and Evil,John Berendt,eng,Intermediate,3.90,167997.0,nonfiction,26182.00,15709.200,...,8.0,Hachette Book Group,3800.0,John Berendt,3.91,679643419,9.78068E+12,388,980,84
3,1905.0,A Little Princess,"Frances Hodgson Burnett, Nancy Bond",eng,Intermediate,4.20,199872.0,genre fiction,23792.34,14275.404,...,10.0,Random House LLC,3666.0,Frances Hodgson Burnett-Nancy Bond,4.19,142437018,9.78014E+12,242,232387,4144
4,1905.0,A Little Princess,"Frances Hodgson Burnett, Nancy Bond",eng,Intermediate,4.20,199872.0,genre fiction,23792.34,14275.404,...,10.0,Random House LLC,3666.0,Frances Hodgson Burnett-Scott McKowen,4.19,1402714548,9.7814E+12,208,491,53


In [16]:
# keep cleaning up
combined.rename(columns={'Book_ratings_count': 'Amazon_ratings_count', 'ratings_count': 'GoodReads_ratings_count'}, inplace=True)



In [17]:
combined.rename(columns={'Author': 'Amazon_Author'})

,Publishing Year,title,Amazon_Author,language_code,Author_Rating,Amazon_average_rating,Amazon_ratings_count,genre,gross sales,publisher revenue,...,sales rank,Publisher,units sold,GoodReads_author,GoodReads_average_rating,isbn,isbn13,# num_pages,GoodReads_ratings_count,text_reviews_count
0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903.0,genre fiction,34160.00,20496.000,...,1.0,HarperCollins Publishers,7000.0,Unknown-Burton Raffel,3.43,451527402,9.78045E+12,160,2735,149
1,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903.0,genre fiction,34160.00,20496.000,...,1.0,HarperCollins Publishers,7000.0,Michael Morpurgo-Michael Foreman,3.85,763632066,9.78076E+12,96,43,7
2,1994.0,Midnight in the Garden of Good and Evil,John Berendt,eng,Intermediate,3.90,167997.0,nonfiction,26182.00,15709.200,...,8.0,Hachette Book Group,3800.0,John Berendt,3.91,679643419,9.78068E+12,388,980,84
3,1905.0,A Little Princess,"Frances Hodgson Burnett, Nancy Bond",eng,Intermediate,4.20,199872.0,genre fiction,23792.34,14275.404,...,10.0,Random House LLC,3666.0,Frances Hodgson Burnett-Nancy Bond,4.19,142437018,9.78014E+12,242,232387,4144
4,1905.0,A Little Princess,"Frances Hodgson Burnett, Nancy Bond",eng,Intermediate,4.20,199872.0,genre fiction,23792.34,14275.404,...,10.0,Random House LLC,3666.0,Frances Hodgson Burnett-Scott McKowen,4.19,1402714548,9.7814E+12,208,491,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13716,NaN,M Is for Magic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Neil Gaiman-Teddy Kristiansen,3.82,61186422,9.78006E+12,260,11317,1060
13717,NaN,Black Orchid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Neil Gaiman-Dave McKean,3.72,930289552,9.78093E+12,160,8710,361
13718,NaN,InterWorld (InterWorld #1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Neil Gaiman-Michael Reaves,3.53,61238961,9.78006E+12,239,14334,1485
13719,NaN,The Faeries' Oracle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Brian Froud-Jessica Macbeth,4.43,743201116,9.78074E+12,224,1550,38


In [18]:
combined.rename(columns={'ratings_count': 'GoodReads_ratings_count'})

,Publishing Year,title,Author,language_code,Author_Rating,Amazon_average_rating,Amazon_ratings_count,genre,gross sales,publisher revenue,...,sales rank,Publisher,units sold,GoodReads_author,GoodReads_average_rating,isbn,isbn13,# num_pages,GoodReads_ratings_count,text_reviews_count
0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903.0,genre fiction,34160.00,20496.000,...,1.0,HarperCollins Publishers,7000.0,Unknown-Burton Raffel,3.43,451527402,9.78045E+12,160,2735,149
1,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903.0,genre fiction,34160.00,20496.000,...,1.0,HarperCollins Publishers,7000.0,Michael Morpurgo-Michael Foreman,3.85,763632066,9.78076E+12,96,43,7
2,1994.0,Midnight in the Garden of Good and Evil,John Berendt,eng,Intermediate,3.90,167997.0,nonfiction,26182.00,15709.200,...,8.0,Hachette Book Group,3800.0,John Berendt,3.91,679643419,9.78068E+12,388,980,84
3,1905.0,A Little Princess,"Frances Hodgson Burnett, Nancy Bond",eng,Intermediate,4.20,199872.0,genre fiction,23792.34,14275.404,...,10.0,Random House LLC,3666.0,Frances Hodgson Burnett-Nancy Bond,4.19,142437018,9.78014E+12,242,232387,4144
4,1905.0,A Little Princess,"Frances Hodgson Burnett, Nancy Bond",eng,Intermediate,4.20,199872.0,genre fiction,23792.34,14275.404,...,10.0,Random House LLC,3666.0,Frances Hodgson Burnett-Scott McKowen,4.19,1402714548,9.7814E+12,208,491,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13716,NaN,M Is for Magic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Neil Gaiman-Teddy Kristiansen,3.82,61186422,9.78006E+12,260,11317,1060
13717,NaN,Black Orchid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Neil Gaiman-Dave McKean,3.72,930289552,9.78093E+12,160,8710,361
13718,NaN,InterWorld (InterWorld #1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Neil Gaiman-Michael Reaves,3.53,61238961,9.78006E+12,239,14334,1485
13719,NaN,The Faeries' Oracle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Brian Froud-Jessica Macbeth,4.43,743201116,9.78074E+12,224,1550,38


In [27]:
# start building the sql dataframe books with tables
conn = mysql.connector.connect(host='localhost',
                               port=3306,
                               database='books',
                               user='root',
                               password='jarebear9')

mycursor = conn.cursor()

In [28]:
mycursor.execute("CREATE TABLE amazon (title VARCHAR(255), Amazon_Author VARCHAR(255))")

In [29]:
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

('amazon',)


In [31]:
# add to amazon table
mycursor.execute("ALTER TABLE amazon ADD COLUMN (Amazon_average_rating INT, Amazon_ratings_count INT, genre VARCHAR(255), gross_sales INT, sales_rank INT)")

In [35]:
mycursor.execute("SHOW COLUMNS FROM amazon")
for x in mycursor:
    print(x)

('title', 'varchar(255)', 'YES', '', None, '')
('Amazon_Author', 'varchar(255)', 'YES', '', None, '')
('Amazon_average_rating', 'int(11)', 'YES', '', None, '')
('Amazon_ratings_count', 'int(11)', 'YES', '', None, '')
('genre', 'varchar(255)', 'YES', '', None, '')
('gross_sales', 'int(11)', 'YES', '', None, '')
('sales_rank', 'int(11)', 'YES', '', None, '')


In [36]:
# add a primary key
mycursor.execute("ALTER TABLE amazon ADD COLUMN id INT AUTO_INCREMENT PRIMARY KEY")

In [37]:
mycursor.execute("SHOW COLUMNS FROM amazon")
for x in mycursor:
    print(x)

('title', 'varchar(255)', 'YES', '', None, '')
('Amazon_Author', 'varchar(255)', 'YES', '', None, '')
('Amazon_average_rating', 'int(11)', 'YES', '', None, '')
('Amazon_ratings_count', 'int(11)', 'YES', '', None, '')
('genre', 'varchar(255)', 'YES', '', None, '')
('gross_sales', 'int(11)', 'YES', '', None, '')
('sales_rank', 'int(11)', 'YES', '', None, '')
('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment')


In [38]:
# add goodreads table
mycursor.execute("CREATE TABLE goodreads (title VARCHAR(255), GoodReads_author VARCHAR(255), GoodReads_average_rating INT, GoodReads_ratings_count INT)")

In [39]:
mycursor.execute("SHOW COLUMNS FROM goodreads")
for x in mycursor:
    print(x)

('title', 'varchar(255)', 'YES', '', None, '')
('GoodReads_author', 'varchar(255)', 'YES', '', None, '')
('GoodReads_average_rating', 'int(11)', 'YES', '', None, '')
('GoodReads_ratings_count', 'int(11)', 'YES', '', None, '')


In [40]:
# add book info table
mycursor.execute("CREATE TABLE book_info (title VARCHAR(255), genre VARCHAR(255), num_pages INT, isbn BIGINT)")

In [41]:
mycursor.execute("SHOW COLUMNS FROM book_info")
for x in mycursor:
    print(x)

('title', 'varchar(255)', 'YES', '', None, '')
('genre', 'varchar(255)', 'YES', '', None, '')
('num_pages', 'int(11)', 'YES', '', None, '')
('isbn', 'bigint(20)', 'YES', '', None, '')


In [42]:
# use sqlite to connect to database and load data
engine = create_engine('sqlite://', echo=False)

In [43]:
df = pd.DataFrame(combined)
df

,Publishing Year,title,Author,language_code,Author_Rating,Amazon_average_rating,Amazon_ratings_count,genre,gross sales,publisher revenue,...,sales rank,Publisher,units sold,GoodReads_author,GoodReads_average_rating,isbn,isbn13,# num_pages,GoodReads_ratings_count,text_reviews_count
0,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903.0,genre fiction,34160.00,20496.000,...,1.0,HarperCollins Publishers,7000.0,Unknown-Burton Raffel,3.43,451527402,9.78045E+12,160,2735,149
1,1975.0,Beowulf,"Unknown, Seamus Heaney",en-US,Novice,3.42,155903.0,genre fiction,34160.00,20496.000,...,1.0,HarperCollins Publishers,7000.0,Michael Morpurgo-Michael Foreman,3.85,763632066,9.78076E+12,96,43,7
2,1994.0,Midnight in the Garden of Good and Evil,John Berendt,eng,Intermediate,3.90,167997.0,nonfiction,26182.00,15709.200,...,8.0,Hachette Book Group,3800.0,John Berendt,3.91,679643419,9.78068E+12,388,980,84
3,1905.0,A Little Princess,"Frances Hodgson Burnett, Nancy Bond",eng,Intermediate,4.20,199872.0,genre fiction,23792.34,14275.404,...,10.0,Random House LLC,3666.0,Frances Hodgson Burnett-Nancy Bond,4.19,142437018,9.78014E+12,242,232387,4144
4,1905.0,A Little Princess,"Frances Hodgson Burnett, Nancy Bond",eng,Intermediate,4.20,199872.0,genre fiction,23792.34,14275.404,...,10.0,Random House LLC,3666.0,Frances Hodgson Burnett-Scott McKowen,4.19,1402714548,9.7814E+12,208,491,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13716,NaN,M Is for Magic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Neil Gaiman-Teddy Kristiansen,3.82,61186422,9.78006E+12,260,11317,1060
13717,NaN,Black Orchid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Neil Gaiman-Dave McKean,3.72,930289552,9.78093E+12,160,8710,361
13718,NaN,InterWorld (InterWorld #1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Neil Gaiman-Michael Reaves,3.53,61238961,9.78006E+12,239,14334,1485
13719,NaN,The Faeries' Oracle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Brian Froud-Jessica Macbeth,4.43,743201116,9.78074E+12,224,1550,38


In [45]:
df.to_sql('titles', con=engine)
engine.execute("SELECT * FROM titles").fetchall()

[(0, 1975.0, 'Beowulf', 'Unknown, Seamus Heaney', 'en-US', 'Novice', 3.42, 155903.0, 'genre fiction', 34160.0, 20496.0, 4.88, 1.0, 'HarperCollins Publishers', 7000.0, 'Unknown-Burton Raffel', '3.43', '451527402', '9.78045E+12', '160', 2735, 149),
 (1, 1975.0, 'Beowulf', 'Unknown, Seamus Heaney', 'en-US', 'Novice', 3.42, 155903.0, 'genre fiction', 34160.0, 20496.0, 4.88, 1.0, 'HarperCollins Publishers', 7000.0, 'Michael Morpurgo-Michael Foreman', '3.85', '763632066', '9.78076E+12', '96', 43, 7),
 (2, 1994.0, 'Midnight in the Garden of Good and Evil', 'John Berendt', 'eng', 'Intermediate', 3.9, 167997.0, 'nonfiction', 26182.0, 15709.2, 6.89, 8.0, 'Hachette Book Group', 3800.0, 'John Berendt', '3.91', '679643419', '9.78068E+12', '388', 980, 84),
 (3, 1905.0, 'A Little Princess', 'Frances Hodgson Burnett, Nancy Bond', 'eng', 'Intermediate', 4.2, 199872.0, 'genre fiction', 23792.34, 14275.403999999999, 6.49, 10.0, 'Random House LLC', 3666.0, 'Frances Hodgson Burnett-Nancy Bond', '4.19', '14

In [46]:
# export df to check merge
df.to_csv(r'C:\Users\emily\Desktop\ETL project\combined_data.csv')